In [1]:
from collections import OrderedDict
import numpy as np
import polars as pl

def ensemble_predictions(file_list):
    # Initialize variables
    data_all = []
    sample_id = None
    columns = None
    koef_sum = sum(koef for _, koef in file_list)

    # Iterate through each file and its corresponding weight
    for file, koef in file_list:
        print(file)
        # Read the data from the file
        df_sub = pl.read_csv(file)

        # Extract column names and sample IDs
        if sample_id is None:
            sample_id = df_sub['sample_id'].to_numpy()
            columns = [col for col in df_sub.schema]

        # Multiply each column (except sample_id) by the weight
        data_all.append(df_sub[columns[1:]].to_numpy() * koef)

    # Calculate the weighted average
    data_all = np.mean(data_all, axis=0) * len(file_list) / koef_sum

    # Create a new DataFrame with the combined predictions
    data_list = [(columns[0], sample_id)] + [(columns[i+1], data_all[:, i]) for i in range(len(columns) - 1)]
    df_sub = pl.DataFrame(dict(data_list))

    return df_sub

# Example usage:
file_list = [
    ("/kaggle/input/keras-baseline-seq2seq/submission.csv", 0.1),
    ("/kaggle/input/k/konstantinboyko/keras-baseline-seq2seq/submission.csv", 0.2),
    ("/kaggle/input/k/bestwater/leap-u-net/submission.csv", 0.25),
    ("/kaggle/input/best-single-model-output-leap/submission.csv", 0.45),
]

# Generate the ensemble prediction
ensemble_df = ensemble_predictions(file_list)

# Write the ensemble prediction to a CSV file
ensemble_df.write_csv("./submission.csv")


/kaggle/input/keras-baseline-seq2seq/submission.csv
/kaggle/input/k/konstantinboyko/keras-baseline-seq2seq/submission.csv
/kaggle/input/k/bestwater/leap-u-net/submission.csv
/kaggle/input/best-single-model-output-leap/submission.csv


In [2]:
ensemble_df

sample_id,ptend_t_0,ptend_t_1,ptend_t_2,ptend_t_3,ptend_t_4,ptend_t_5,ptend_t_6,ptend_t_7,ptend_t_8,ptend_t_9,ptend_t_10,ptend_t_11,ptend_t_12,ptend_t_13,ptend_t_14,ptend_t_15,ptend_t_16,ptend_t_17,ptend_t_18,ptend_t_19,ptend_t_20,ptend_t_21,ptend_t_22,ptend_t_23,ptend_t_24,ptend_t_25,ptend_t_26,ptend_t_27,ptend_t_28,ptend_t_29,ptend_t_30,ptend_t_31,ptend_t_32,ptend_t_33,ptend_t_34,ptend_t_35,…,ptend_v_31,ptend_v_32,ptend_v_33,ptend_v_34,ptend_v_35,ptend_v_36,ptend_v_37,ptend_v_38,ptend_v_39,ptend_v_40,ptend_v_41,ptend_v_42,ptend_v_43,ptend_v_44,ptend_v_45,ptend_v_46,ptend_v_47,ptend_v_48,ptend_v_49,ptend_v_50,ptend_v_51,ptend_v_52,ptend_v_53,ptend_v_54,ptend_v_55,ptend_v_56,ptend_v_57,ptend_v_58,ptend_v_59,cam_out_NETSW,cam_out_FLWDS,cam_out_PRECSC,cam_out_PRECC,cam_out_SOLS,cam_out_SOLL,cam_out_SOLSD,cam_out_SOLLD
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""test_0""",0.000004,-0.000062,-0.000082,-0.000085,-0.000095,-0.000097,-0.000078,-0.000062,-0.000047,-0.000039,-0.000028,-0.000017,-0.000008,-0.000004,-3.9162e-7,0.000002,0.000005,0.000004,0.000003,-0.000007,0.000007,-0.000002,-0.000012,-0.000017,-0.000014,-0.000013,-0.000015,-0.000017,-0.000023,-0.000029,-0.000035,-0.000031,-0.000024,-0.00002,-0.000019,-0.000017,…,0.000002,0.000003,-1.7207e-7,-0.000002,-0.000001,-8.0385e-7,-7.0076e-7,-0.000004,-0.000001,-6.5748e-7,2.6321e-7,-0.000001,0.000004,0.000002,8.9752e-8,-0.000005,-0.000008,-0.000026,-0.000044,-0.000043,-0.000021,0.000035,0.000042,0.000043,0.000023,0.000013,0.000009,0.000007,5.0342e-8,-6.621972,396.91939,1.3585e-10,1.2311e-10,-2.799919,-3.736818,-0.810049,-1.042609
"""test_10""",-0.000016,-0.00005,-0.000034,-0.000053,-0.000094,-0.000133,-0.000136,-0.000092,-0.000045,-0.000026,-0.000018,-0.000014,-0.000011,-0.000009,-0.000006,-0.000005,-0.000004,-0.000003,-0.000002,-0.000004,-0.000004,-0.000007,-0.000008,-0.000008,-0.000009,-0.000011,-0.000014,-0.000017,-0.000021,-0.000023,-0.000022,-0.000021,-0.000014,-0.000004,0.000002,0.000007,…,-8.9079e-7,-6.2981e-7,9.4269e-8,7.2209e-7,6.5768e-7,3.3520e-7,2.8603e-7,1.5525e-7,1.7160e-7,6.7476e-8,1.2456e-7,-8.9119e-7,1.1540e-7,-2.3549e-7,0.000002,-2.1591e-7,-8.3076e-7,-0.000002,-0.000003,-0.000002,-0.000002,-0.000002,-4.8478e-7,0.000001,0.000003,0.000003,0.000001,4.8234e-9,0.000005,4.554864,335.947308,3.0210e-10,2.3809e-9,-2.606503,-1.805405,0.703089,2.066207
"""test_100""",-0.000006,-0.000074,-0.000033,-0.000026,-0.000069,-0.000093,-0.000077,-0.000052,-0.00003,-0.000021,-0.000016,-0.000014,-0.000012,-0.000009,-0.000007,-0.000005,-0.000004,-0.000004,0.000002,-0.000007,-0.000005,-0.000007,-0.000006,-0.000006,-0.000005,-0.000005,-0.000007,-0.000009,-0.00001,-0.000012,-0.000014,-0.000017,-0.000019,-0.00002,-0.000021,-0.000022,…,-0.000002,-0.000002,-0.000001,-2.9406e-7,2.4069e-7,4.4812e-7,7.5337e-7,9.5564e-7,9.5605e-7,6.8991e-7,3.9313e-7,-1.9616e-7,4.0406e-7,0.000002,8.8719e-7,-1.9046e-7,-3.9685e-7,-0.000005,-0.000011,-0.000021,-0.000024,0.000011,0.00001,0.000016,0.000012,0.000005,0.000005,0.000003,0.000012,-5.762874,329.60313,-1.1461e-10,6.7594e-10,-3.40861,-0.353193,0.223945,0.02872
"""test_1000""",0.000027,0.000008,0.000032,0.000028,-0.000002,-0.00005,-0.000048,-0.000022,-0.000007,-0.000004,-0.000001,-0.000001,-2.1932e-7,0.000001,0.000003,0.000005,0.000006,0.000005,5.2849e-7,-5.3340e-7,8.5094e-8,-6.6687e-7,-0.00001,-0.000018,-0.000017,-0.000013,-0.000014,-0.000017,-0.000021,-0.000023,-0.000021,-0.000021,-0.000018,-0.000016,-0.000015,-0.000017,…,1.2958e-7,-9.6295e-8,8.0705e-7,-0.000001,-1.5428e-10,0.000002,-0.000002,3.9853e-7,-0.000001,3.1565e-7,-0.000005,0.000008,0.000006,0.000006,0.000008,0.000006,0.000006,0.000008,0.000006,0.000004,0.000003,7.9901e-7,-9.9315e-7,-0.000006,-0.00001,-0.000018,-0.000031,-0.000043,0.000027,1.248324,3